I am encountering some issues getting my `Local_Join_Count_BV` to agree with the output from GeoDa. In this notebook I will try to present the problem as clearly as possible. 

Firstly, when comparing results to GeoDa, we note that:

> In GeoDa, the co-location case is treated under Multivariate Local Join Count (which includes the bivariate setup as a special case), whereas the no co-location case is implemented under Bivariate Local Join Count.

Let's begin by loading in the trial data. We use the [Baltimore Housing Sales dataset](https://geodacenter.github.io/data-and-lab/baltim/) and focus on the 'dwell' ($x$) and 'patio' ($z$) binary variables. 

In [1]:
import geopandas as gpd
balt = gpd.read_file('https://github.com/jeffcsauer/GSOC2020/raw/master/validation/data/baltimore/baltimore_housing.gpkg')
balt.head()

,station,price,nroom,dwell,nbath,patio,firepl,ac,bment,nstor,gar,age,citcou,lotsz,sqft,x,y,geometry
0,1.0,47.0,4.0,0.0,1.0,0.0,0.0,0.0,2.0,3.0,0.0,148.0,0.0,5.70,11.25,907.0,534.0,POINT (907.000 534.000)
1,2.0,113.0,7.0,1.0,2.5,1.0,1.0,1.0,2.0,2.0,2.0,9.0,1.0,279.51,28.92,922.0,574.0,POINT (922.000 574.000)
2,3.0,165.0,7.0,1.0,2.5,1.0,1.0,0.0,3.0,2.0,2.0,23.0,1.0,70.64,30.62,920.0,581.0,POINT (920.000 581.000)
3,4.0,104.3,7.0,1.0,2.5,1.0,1.0,1.0,2.0,2.0,2.0,5.0,1.0,174.63,26.12,923.0,578.0,POINT (923.000 578.000)
4,5.0,62.5,7.0,1.0,1.5,1.0,1.0,0.0,2.0,2.0,0.0,19.0,1.0,107.80,22.04,918.0,574.0,POINT (918.000 574.000)


Isolate the variable of interest.

In [2]:
x_balt = balt['dwell']
z_balt = balt['patio']

Create knn (n=5) weights object.

In [3]:
points = list(zip(balt['x'], balt['y']))
import libpysal
import numpy as np
kd = libpysal.cg.KDTree(np.array(points))
balt_knn5 = libpysal.weights.KNN(kd, k=5) 

We now define and apply the `Local_Join_Count_BV` function.

In [4]:
import pandas as pd
import warnings
from scipy import sparse
from sklearn.base import BaseEstimator
from libpysal import weights

PERMUTATIONS = 999

class Local_Join_Count_BV(BaseEstimator):

    """Bivariate local join counts"""

    def __init__(self, connectivity=None, permutations=PERMUTATIONS):
        """
        Initialize a Local_Join_Count_BV estimator
        Arguments
        ---------
        connectivity:   scipy.sparse matrix object
                        the connectivity structure describing the relationships
                        between observed units. Will be row-standardized.
        Attributes
        ----------
        LJC       :   numpy.ndarray
                      array containing the estimated Bivariate Local Join Counts
        p_sim       :   numpy.ndarray
                        array containing the simulated p-values for each unit.
        """

        self.connectivity = connectivity
        self.permutations = permutations

    def fit(self, x, z, case="CLC", permutations=999):
        """
        Arguments
        ---------
        x       :   numpy.ndarray
                    array containing binary (0/1) data
        y       :   numpy.ndarray
                    array containing binary (0/1) data
        Returns
        -------
        the fitted estimator.
        Notes
        -----
        Technical details and derivations can be found in :cite:`AnselinLi2019`.
        """
        x = np.asarray(x).flatten()
        z = np.asarray(z).flatten()

        w = self.connectivity
        # Fill the diagonal with 0s
        w = weights.util.fill_diagonal(w, val=0)
        w.transform = 'b'

        self.x = x
        self.z = z
        self.n = len(x)
        self.w = w
        self.case = case

        self.LJC = self._statistic(x, z, w, case=case)

        if permutations:
            self._crand()
            sim = np.transpose(self.rjoins)
            above = sim >= self.LJC
            larger = above.sum(0)
            low_extreme = (self.permutations - larger) < larger
            larger[low_extreme] = self.permutations - larger[low_extreme]
            self.p_sim = (larger + 1.0) / (permutations + 1.0)
            # Set p-values for those with LJC of 0 to NaN
            self.p_sim[self.LJC==0] = 'NaN'

        return self

    @staticmethod
    def _statistic(x, z, w, case):
        # Create adjacency list. Note that remove_symmetric=False - this is
        # different from the esda.Join_Counts() function.
        adj_list = w.to_adjlist(remove_symmetric=False)

        # First, set up a series that maps the values
        # to the weights table
        zseries_x = pd.Series(x, index=w.id_order)
        zseries_z = pd.Series(z, index=w.id_order)

        # Map the values to the focal (i) values
        focal_x = zseries_x.loc[adj_list.focal].values
        focal_z = zseries_z.loc[adj_list.focal].values

        # Map the values to the neighbor (j) values
        neighbor_x = zseries_x.loc[adj_list.neighbor].values
        neighbor_z = zseries_z.loc[adj_list.neighbor].values

        if case == "BJC":
            BJC = (focal_x == 1) & (focal_z == 0) & \
                  (neighbor_x == 0) & (neighbor_z == 1)
            adj_list_BJC = pd.DataFrame(adj_list.focal.values,
                                        BJC.astype('uint8')).reset_index()
            adj_list_BJC.columns = ['BJC', 'ID']
            adj_list_BJC = adj_list_BJC.groupby(by='ID').sum()
            return (adj_list_BJC.BJC.values)
        elif case == "CLC":
            CLC = (focal_x == 1) & (focal_z == 1) & \
                  (neighbor_x == 1) & (neighbor_z == 1)
            adj_list_CLC = pd.DataFrame(adj_list.focal.values,
                                        CLC.astype('uint8')).reset_index()
            adj_list_CLC.columns = ['CLC', 'ID']
            adj_list_CLC = adj_list_CLC.groupby(by='ID').sum()
            return (adj_list_CLC.CLC.values)
        else:
            raise NotImplementedError(f'The requested LJC method ({case}) is not currently supported!')
            
    def _crand(self):
        """
        conditional randomization

        for observation i with ni neighbors,  the candidate set cannot include
        i (we don't want i being a neighbor of i). we have to sample without
        replacement from a set of ids that doesn't include i. numpy doesn't
        directly support sampling wo replacement and it is expensive to
        implement this. instead we omit i from the original ids,  permute the
        ids and take the first ni elements of the permuted ids as the
        neighbors to i in each randomization.

        """
        x = self.x
        z = self.z
        xz = ((x==1) & (z==0)).astype('uint8')
        case = self.case
        
        n = len(x)
        joins = np.zeros((self.n, self.permutations))
        n_1 = self.n - 1
        prange = list(range(self.permutations))
        k = self.w.max_neighbors + 1
        nn = self.n - 1
        rids = np.array([np.random.permutation(nn)[0:k] for i in prange])
        ids = np.arange(self.w.n)
        ido = self.w.id_order
        w = [self.w.weights[ido[i]] for i in ids]
        wc = [self.w.cardinalities[ido[i]] for i in ids]

        for i in range(self.w.n):
            idsi = ids[ids != i]
            np.random.shuffle(idsi)
            tmp_x = x[idsi[rids[:, 0:wc[i]]]]
            tmp_z = z[idsi[rids[:, 0:wc[i]]]]
            tmp_xz = xz[idsi[rids[:, 0:wc[i]]]]
            if case == "BJC":
                joins[i] = z[i] * (w[i] * tmp_xz).sum(1)
            elif case == "CLC":
                joins[i] = z[i] * (w[i] * tmp_z * tmp_x).sum(1)
            else:
                raise NotImplementedError(f'The requested LJC method ({case}) is not currently supported!')
        self.rjoins = joins

In [5]:
# Test results for Case 1
test_results_case1 = Local_Join_Count_BV(connectivity=balt_knn5).fit(x_balt, z_balt, case="BJC")

# Test results for Case 2
test_results_case2 = Local_Join_Count_BV(connectivity=balt_knn5).fit(x_balt, z_balt, case="CLC")

Compare our results to those of GeoDa.

In [6]:
# Load GeoDa analysis results
GeoDa_LJC = pd.read_csv('https://github.com/jeffcsauer/GSOC2020/raw/master/validation/data/baltimore/balt_knn_5_LJC_bivariate.csv')
GeoDa_LJC.head()

,station,price,nroom,dwell,nbath,patio,firepl,ac,bment,nstor,...,lotsz,sqft,x,y,JC_C1,NN_C1,PP_VAL_C1,JC_C2,NN_C2,PP_VAL_C2
0,1,47.0,4.0,0.0,1.0,0.0,0.0,0.0,2.0,3.0,...,5.70,11.25,907.0,534.0,0,5,NaN,0,5,NaN
1,2,113.0,7.0,1.0,2.5,1.0,1.0,1.0,2.0,2.0,...,279.51,28.92,922.0,574.0,4,5,0.001,4,5,0.001
2,3,165.0,7.0,1.0,2.5,1.0,1.0,0.0,3.0,2.0,...,70.64,30.62,920.0,581.0,5,5,0.001,5,5,0.001
3,4,104.3,7.0,1.0,2.5,1.0,1.0,1.0,2.0,2.0,...,174.63,26.12,923.0,578.0,5,5,0.001,5,5,0.001
4,5,62.5,7.0,1.0,1.5,1.0,1.0,0.0,2.0,2.0,...,107.80,22.04,918.0,574.0,3,5,0.015,3,5,0.015


This is where things get a little interesting. We note that the number of join counts (column `JC_C1` for Case 1 and `JC_C2` for Case 2) seem to be the same. Let's see how our function compares. 

In [7]:
print("Comparison of GeoDa bivariate LJC to PySAL implementation (Case 1):")
results = test_results_case1.LJC == GeoDa_LJC['JC_C1']
print(results.value_counts())
print("--------------------------")
print("Comparison of GeoDa bivariate LJC to PySAL implementation (Case 2):")
results = test_results_case2.LJC == GeoDa_LJC['JC_C2']
print(results.value_counts())

Comparison of GeoDa bivariate LJC to PySAL implementation (Case 1):
True     179
False     32
Name: JC_C1, dtype: int64
--------------------------
Comparison of GeoDa bivariate LJC to PySAL implementation (Case 2):
True    211
Name: JC_C2, dtype: int64


We get a 100% match for Case 2, but only a ~85% match for Case 1. Let's find some instances where Case 1 disagrees and see if we can figure out what is going on.

In [8]:
test_results_case1.LJC == GeoDa_LJC['JC_C1']

0       True
1      False
2      False
3      False
4      False
       ...  
206     True
207     True
208     True
209     True
210     True
Name: JC_C1, Length: 211, dtype: bool

We will use areal unit `1` from above as our test subject. Let's get the neighbors for that unit.  

In [9]:
balt_knn5[1]

{4: 1.0, 3: 1.0, 6: 1.0, 184: 1.0, 14: 1.0}

Based on the premise of Case 1, we are looking for instances where: $x_i=1$ and $z_i = 0$, and $x_j = 0$ when $z_j = 1$. Let's consider the focal ($_i$) values of $x$ and $z$ for this unit. 

In [10]:
x_balt[1]

1.0

In [11]:
z_balt[0]

0.0

We note that it appears our focal values meet the condition of $x_i=1$ and $z_i = 0$. 

Now, we want to see how many neighbors meet the condition of $x_j = 0$ when $z_j = 1$. To do so, we simply print out the associated value at each of the neighbor indices for $x$ and $z$.

In [12]:
print(x_balt[4], x_balt[3], x_balt[6], x_balt[184], x_balt[14])

1.0 1.0 1.0 1.0 0.0


In [13]:
print(z_balt[4], z_balt[3], z_balt[6], z_balt[184], z_balt[14])

1.0 1.0 1.0 1.0 0.0


We note that there does not seem to be any neighbors that meet the condition of $x_j = 0$ when $z_j = 1$. To my understanding we would expect the Case 1 Bivariate LJC for this unit to be 0. However, GeoDa reports this as:

In [14]:
GeoDa_LJC['JC_C1'][1]

4

My function reports it as: 

In [15]:
test_results_case1.LJC[1]

0

# Trying the function on the `commpop` dataset

With additional strangeness, we get the opposite scenario when trying another dataset. This is the `commpop` dataset and already includes results from GeoDa. The variables of interest are popneg ($x$) and popplus ($y$).

In [16]:
commpop = gpd.read_file("https://github.com/jeffcsauer/GSOC2020/raw/master/validation/data/commpop_geodavalues.gpkg")
commpop.head()

,POLY_ID,community,NID,POP2010,POP2000,POPCH,POPPERCH,popplus,popneg,JC_BV_C1,NN_BV_C1,PP_VAL_BV_C1,JC_BV_C2,NN_BV_C2,PP_VAL_BV_C2,geometry
0,1,DOUGLAS,35.0,18238.0,26470.0,-8232.0,-31.099358,0.0,1.0,2,4,0.213,0,4,NaN,"MULTIPOLYGON (((-87.60914 41.84469, -87.60915 ..."
1,2,OAKLAND,36.0,5918.0,6110.0,-192.0,-3.142390,0.0,1.0,0,3,NaN,0,3,NaN,"MULTIPOLYGON (((-87.59215 41.81693, -87.59231 ..."
2,3,FULLER PARK,37.0,2876.0,3420.0,-544.0,-15.906433,0.0,1.0,1,6,0.200,0,6,NaN,"MULTIPOLYGON (((-87.62880 41.80189, -87.62879 ..."
3,4,GRAND BOULEVARD,38.0,21929.0,28006.0,-6077.0,-21.698922,0.0,1.0,1,7,0.156,0,7,NaN,"MULTIPOLYGON (((-87.60671 41.81681, -87.60670 ..."
4,5,KENWOOD,39.0,17841.0,18363.0,-522.0,-2.842673,0.0,1.0,0,4,NaN,0,4,NaN,"MULTIPOLYGON (((-87.59215 41.81693, -87.59215 ..."


GeoDa results are `JC_BV_C1` (Case 1) and `JC_BV_C2` (Case 2). We use queen weights.

In [17]:
qw_comm = libpysal.weights.Queen.from_dataframe(commpop)

In [18]:
# Test results for Case 1
test_results_case1 = Local_Join_Count_BV(connectivity=qw_comm).fit(commpop['popneg'], commpop['popplus'], case="BJC")

# Test results for Case 2
test_results_case2 = Local_Join_Count_BV(connectivity=qw_comm).fit(commpop['popneg'],  commpop['popplus'], case="CLC")

In [19]:
print("Comparison of GeoDa bivariate LJC to PySAL implementation (Case 1):")
results = test_results_case1.LJC == commpop['JC_BV_C1']
print(results.value_counts())
print("--------------------------")
print("Comparison of GeoDa bivariate LJC to PySAL implementation (Case 2):")
results = test_results_case2.LJC == commpop['JC_BV_C2']
print(results.value_counts())

Comparison of GeoDa bivariate LJC to PySAL implementation (Case 1):
True    77
Name: JC_BV_C1, dtype: int64
--------------------------
Comparison of GeoDa bivariate LJC to PySAL implementation (Case 2):
True     61
False    16
Name: JC_BV_C2, dtype: int64


Now it seems the Case 2 counts are off! This is even more confounding as the units with disagreement do not seem to meet the CLC conditions. For example, let's examine unit 72. 

In [20]:
test_results_case2.LJC == commpop['JC_BV_C2']

0      True
1      True
2      True
3      True
4      True
      ...  
72    False
73     True
74    False
75     True
76     True
Name: JC_BV_C2, Length: 77, dtype: bool

We are interested in situations where $x_i = z_i = 1$ as well as $x_j = z_j = 1$. 

In [21]:
commpop.popneg[72]

0.0

In [22]:
commpop.popplus[72]

1.0

Immediately we see that $x_i$ at 72 does not equal 1. GeoDa reports the Case 2 Bivarate LJC for this unit as: 

In [23]:
commpop.JC_BV_C2[72]

2

My function reports it as:

In [24]:
test_results_case2.LJC[72]

0